In [31]:
import sys
import re
import gzip
from tqdm import tqdm

def get_line_number(file_path):
    sys.stderr.write("Counting line number of {}".format(file_path))

    f = gzip.open(file_path)
    lines = f.readlines()
    count =0
    
    for line in lines:
        count += 1
    return count

def parse(filename, total):
    IGNORE_FIELDS = ['Total items', 'reviews']
    f = open(filename, 'r')
    lines = f.readlines()
    entry = {}
    categories = []
    reviews = []
    similar_items = []
  
    for line in lines[0:total]:
        colonPos = line.find(':')

        if line.startswith("Id"):
            if reviews:
                entry["reviews"] = reviews
            if categories:
                entry["categories"] = categories
            yield entry
            entry = {}
            categories = []
            reviews = []
            rest = line[colonPos+2:]
            entry["id"] = rest[1:-1]
      
        elif line.startswith("similar"):
            similar_items = line.split()[2:]
            entry['similar_items'] = similar_items

    # "cutomer" is typo of "customer" in original data
        elif line.find("cutomer:") != -1:
            review_info = line.split()
            reviews.append({'customer_id': review_info[2], 
                          'rating': int(review_info[4]), 
                          'votes': int(review_info[6]), 
                          'helpful': int(review_info[8])})

        elif line.startswith("|"):
            categories.append(line)

        elif colonPos != -1:
            eName = line[:colonPos]
            rest = line[colonPos+2:]
            if not eName in IGNORE_FIELDS:
                if eName[0] == ' ':
                    eName = eName[2:]
                entry[eName] = rest[0:-1]

    if reviews:
        entry["reviews"] = reviews
    if categories:
        entry["categories"] = categories
    
    yield entry


if __name__ == '__main__':
    file_path = "data/amazon-meta.txt"
#     line_num = get_line_number(file_path)
#     print(line_num)/
    for e in parse(file_path, total=226):
        if e:
            print((e))
#             print(simplejson.dumps(e))


{'id': ' 0', 'ASIN': '0771044445'}
{'id': ' 1', 'ASIN': '0827229534', 'title': 'Patterns of Preaching: A Sermon Sampler', 'group': 'Book', 'salesrank': '396585', 'similar': '5  0804215715  156101074X  0687023955  0687074231  082721619X', 'categories': '2', 'reviews': [{'customer_id': 'A2JW67OY8U6HHK', 'rating': 5, 'votes': 10, 'helpful': 9}, {'customer_id': 'A2VE83MZF98ITY', 'rating': 5, 'votes': 6, 'helpful': 5}]}
{'id': ' 2', 'ASIN': '0738700797', 'title': 'Candlemas: Feast of Flames', 'group': 'Book', 'salesrank': '168596', 'similar': '5  0738700827  1567184960  1567182836  0738700525  0738700940', 'categories': '2', 'reviews': [{'customer_id': 'A11NCO6YTE4BTJ', 'rating': 5, 'votes': 5, 'helpful': 4}, {'customer_id': 'A9CQ3PLRNIR83', 'rating': 4, 'votes': 5, 'helpful': 5}, {'customer_id': 'A13SG9ACZ9O5IM', 'rating': 5, 'votes': 8, 'helpful': 8}, {'customer_id': 'A1BDAI6VEYMAZA', 'rating': 5, 'votes': 4, 'helpful': 4}, {'customer_id': 'A2P6KAWXJ16234', 'rating': 4, 'votes': 16, 'help